In [131]:
import requests
import json

In [132]:
def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    #response = requests.post(automat_url,json=message['message'])
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        return {}
    #print(response.json())
    return response.json()

def strider(message):
    #url = 'http://robokop.renci.org:5781/query'
    url = 'http://robokop.renci.org:5781/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    #numanswers = len(strider_answer['results'])
    #if (numanswers == 0) or ((numanswers == 1) and (len(strider_answer['results'][0]['node_bindings'])==0)):
    #    print('no answers')
    #    return {}
    #Strider for some reason doesn't return the query graph
    #strider_answer['query_graph'] = message['message']['query_graph']
    return strider_answer
 
def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/graph',{'message':strider_answer})
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',{'message': coalesced_answer})
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',{'message': omni_answer})
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',{'message': weighted_answer})
    return scored_answer

def aragorn(message, coalesce_type):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

In [133]:
#message = one_hop_message('MONDO:0005090','disease','chemical_substance','treats',reverse=True)
message = {
  "message": {
    "query_graph": {
      "nodes": {
        "n00": {
          "id": "MONDO:0005784",
          "category": "biolink:Disease"
        },
        "n01": {
          "category": "biolink:Disease"
        }
      },
      "edges": {
        "e00": {
          "subject": "n00",
          "object": "n01",
          "predicate":"biolink:subclass_of"
        }
      }
    }
  }
}


In [134]:
s = strider(message)

In [135]:
s

{'message': {'query_graph': {'nodes': {'n00': {'id': 'MONDO:0005784',
     'category': 'biolink:Disease',
     'is_set': False},
    'n01': {'id': None, 'category': 'biolink:Disease', 'is_set': False}},
   'edges': {'e00': {'subject': 'n00',
     'object': 'n01',
     'predicate': 'biolink:subclass_of',
     'relation': None}}},
  'knowledge_graph': {'nodes': {'MONDO:0005784': {'category': 'biolink:Disease',
     'name': 'hantavirus hemorrhagic fever with renal syndrome',
     'attributes': [{'name': 'equivalent_identifiers',
       'value': {'id': {'label': 'hantavirus hemorrhagic fever with renal syndrome',
         'identifier': 'MONDO:0005784'},
        'db_ids': {'MONDO': ['MONDO:0005784'],
         'DOID': ['DOID:11266'],
         'EFO': ['EFO:0007299'],
         'MESH': ['D006480'],
         'name': ['hantavirus hemorrhagic fever with renal syndrome']},
        'type': 'Disease',
        'curies': ['MONDO:0005784',
         'DOID:11266',
         'EFO:0007299',
         'MESH:D0